In [ ]:
from typing import Iterable, List, Optional
import datetime
import functools
import operator
import pyinterp.backends.xarray
import os
import pandas as pd
import xarray as xr
import numpy as np
import dask.distributed
import dask_jobqueue

In [ ]:
cluster = dask_jobqueue.PBSCluster(cores=1,
                                   memory='2GB',
                                   interface='ib0',
                                   local_directory="$TMPDIR",
                                   walltime='12:00:00')
cluster.scale(10)
cluster

In [ ]:
client = dask.distributed.Client(cluster)
client

In [ ]:
# Root directory containing the datasets to be processed.
ROOT = "/home/ad/briolf/odatis/briolf/bigdata4science"

# File describing the queries to be executed .
REQUEST = "/home/ad/briolf/notebooks/bigdata4bigscience/requests.csv"

# File containing the results of the benchmarks
RESULT = "/home/ad/briolf/notebooks/bigdata4bigscience/interp.csv"

# Temporary directory
TMPDIR = "/work/ALT/odatis/briolf/tmp"

# Directory containing tracks data
TRACKS = os.path.join(ROOT, "tracks")

In [ ]:
SCENARIO_1 = [
    "NorthSea.csv", 
    "Rotterdam12H.csv", 
    "Rotterdam15m.csv", 
    "Rotterdam1H.csv", 
    "Rotterdam1m.csv", 
    "RotterdamAll.csv", 
]

SCENARIO_2 = [
    "World10D.csv", 
    "World1.csv", 
    "World1D.csv", 
    "World20D.csv", 
    "World50D.csv", 
    "World5D.csv"
]

In [ ]:
def load_request(path: str) -> pd.DataFrame:
    """Loading the track"""
    result = pd.read_csv(path,
                         sep=";",
                         dtype={
                             "locDate": "str",
                             "locTime": "str",
                             "lon": "float64",
                             "lat": "float64",
                         },
                         parse_dates=False,
                         usecols=[0, 1, 2, 3],
                         engine='c')
    dates = (
        result['locDate'].apply(lambda x: x[-4:] + "-" + x[3:5] + "-" + x[:2])
        + "T" + result['locTime']).values
    result["datetime"] = dates.astype("datetime64")
    result.drop("locDate", axis=1, inplace=True)
    result.drop("locTime", axis=1, inplace=True)
    result.sort_values(by=['datetime'], inplace=True, ignore_index=True)
    return result

In [ ]:
def get_dataset_path(name: str) -> str:
    """Gets the dataset path"""
    return os.path.join(ROOT, name)

In [ ]:
def varname_from_standard_name(ds: xr.Dataset,
                               standard_names: str) -> str:
    """Get variable names from standard names."""
    for name, data_array in ds.data_vars.items():
        if data_array.attrs["standard_name"] in standard_names:
            return name
    raise ValueError(f"no such variable: {standard_names}")

In [ ]:
def interpolate_(ds: xr.DataArray,
                 df: pd.DataFrame,
                 name: str,
                 depth: Optional[float] = None) -> pd.Series:
    dt = set(np.diff(ds["time"]))
    if len(dt) != 1:
        raise RuntimeError("The deltaT between two grids is not constant.")
    dt = dt.pop()

    dx = np.diff(ds["longitude"]).max()
    dy = np.diff(ds["latitude"]).max()

    x0 = df.lon.min() - dx
    x1 = df.lon.max() + dx

    y0 = df.lat.min() - dy
    y1 = df.lat.max() + dy
    
    t0 = df.datetime.min()
    t1 = df.datetime.max()
    
    it = ds.sel({'time': t0}, method='nearest').time.data
    t0 = it - dt if it > t0 else t0
    
    it = ds.sel({'time': t1}, method='nearest').time.data
    t1 = it + dt if it < t1 else t1

    isel = {
        "longitude": (ds["longitude"] >= x0) & (ds["longitude"] <= x1),
        "latitude": (ds["latitude"] >= y0) & (ds["latitude"] <= y1),
        "time": (ds["time"] >= t0) & (ds["time"] <= t1)
    }

    if depth is not None:
        isel["depth"] = ds["depth"] == depth

    # Creation of the calculation graph performing the query
    selected = ds.isel(isel)
    if not functools.reduce(operator.mul, selected.shape):
        raise RuntimeError(f"invalid query: {t0}, {t1}")
    if depth is not None:
        selected = selected.squeeze("depth")

    selected = selected.compute()
    interpolator = pyinterp.backends.xarray.RegularGridInterpolator(selected)
    data = interpolator(dict(longitude=df.lon,
                             latitude=df.lat,
                             time=df.datetime.values),
                        method="inverse_distance_weighting",
                        bounds_error=False,
                        num_threads=0)
    return pd.Series(data, df.index, name=name)

In [ ]:
def interpolate(dataset: str,
                df: pd.DataFrame,
                name: str,
                depth: Optional[float] = None) -> None:
    """Runs benchmarks on a given dataset"""
    ds = xr.open_zarr(get_dataset_path(dataset))
    period_start = df.groupby(df.datetime.dt.date).count().index

    # fill variable to interpolate
    df[name] = float("nan")

    # Calculates the period required to interpolate the data from the provided
    # time series
    end = None
    periods = []
    for start, end in zip(period_start, period_start[1:]):
        start = pd.Timestamp(start)
        end = pd.Timestamp(end)
        periods.append([start, end])
    if end is None:
        end = pd.Timestamp(period_start[0])

    # As the last date is excluded from the interval, a second is added to
    # include it in the processing.
    periods.append([end, df.datetime.iloc[-1] + datetime.timedelta(seconds=1)])

    # Finally, the data on the different periods identified are interpolated.
    futures = []
    for start, end in periods:
        varname = varname_from_standard_name(ds, name)
        df_ = client.scatter(df[(df.datetime > start)
                                & (df.datetime < end)])
        futures.append(
            client.submit(interpolate_, ds[varname], df_, name, depth))
        #result = interpolate_(ds[varname], df[(df.datetime > start) & (df.datetime < end)], name, depth)
    for item in dask.distributed.as_completed(futures):
        series = item.result()
        df.loc[series.index, series.name] = series.values

In [ ]:
results = pd.DataFrame(columns=[
    "csv", "dataset", "variable", "average", "best", "worst", "stdev", "loop",
    "repeat", "cores"
])
results

In [ ]:
CASES_1 = {
    "global-analysis-forecast-phy-001-024":
    [("sea_surface_height_above_geoid", None)],
}

CASES_2 = {
    "global-analysis-forecast-phy-001-024": [
        ("sea_surface_height_above_geoid", None),
        ("ocean_mixed_layer_thickness_defined_by_sigma_theta", None),
        ("sea_water_potential_temperature", 5.078224),
        ("eastward_sea_water_velocity", 5.078224),
        ("northward_sea_water_velocity", 5.078224),
        ("sea_water_salinity", 5.078224),
        ("sea_water_potential_temperature_at_sea_floor", None),
        ("sea_surface_height_above_geoid", None),
    ]
}

In [ ]:
for csv in SCENARIO_1:
    track = load_request(os.path.join(TRACKS, csv))
    for dataset, cases in CASES_1.items():
        for name, depth in cases:
            timer = %timeit -r 3 -n 1 -o interpolate(dataset, track, name, depth=depth)
            results = results.append(
                pd.DataFrame(data=[[
                    csv, dataset, name, timer.average, timer.best, timer.worst,
                    timer.stdev, 3, 1, 10
                ]],
                             columns=[
                                 "csv", "dataset", "variable", "average",
                                 "best", "worst", "stdev", "loop", "repeat", "cores"
                             ]))

In [ ]:
results

In [ ]:
for csv in SCENARIO_2:
    track = load_request(os.path.join(TRACKS, csv))
    for dataset, cases in CASES_2.items():
        for name, depth in cases:
            timer = %timeit -r 3 -n 1 -o interpolate(dataset, track, name, depth=depth)
            results = results.append(
                pd.DataFrame(data=[[
                    csv, dataset, name, timer.average, timer.best, timer.worst,
                    timer.stdev, 3, 1, 10
                ]],
                             columns=[
                                 "csv", "dataset", "variable", "average",
                                 "best", "worst", "stdev", "loop", "repeat", "cores"
                             ]))

In [ ]:
csv = "World50D.csv"
track = load_request(os.path.join(TRACKS, csv))

In [ ]:
cluster.scale(20)

In [ ]:
for dataset, cases in CASES_2.items():
    for name, depth in cases:
        timer = %timeit -r 3 -n 1 -o interpolate(dataset, track, name, depth=depth)
        results = results.append(
            pd.DataFrame(data=[[
                csv, dataset, name, timer.average, timer.best, timer.worst,
                timer.stdev, 3, 1, 20
            ]],
                         columns=[
                             "csv", "dataset", "variable", "average",
                             "best", "worst", "stdev", "loop", "repeat", "cores"
                         ]))

In [ ]:
cluster.scale(40)

In [ ]:
for dataset, cases in CASES_2.items():
    for name, depth in cases:
        timer = %timeit -r 3 -n 1 -o interpolate(dataset, track, name, depth=depth)
        results = results.append(
            pd.DataFrame(data=[[
                csv, dataset, name, timer.average, timer.best, timer.worst,
                timer.stdev, 3, 1, 40
            ]],
                         columns=[
                             "csv", "dataset", "variable", "average",
                             "best", "worst", "stdev", "loop", "repeat", "cores"
                         ]))

In [ ]:
results.to_csv(RESULT, sep=";", date_format="%Y-%m-%d %H:%M:%S.000", index=False)

In [29]:
client.close()
cluster.close()